<br>

###   hw7pr1iris_modeler 

Here, you'll epxlore iris clasification <font color="Coral">and regression</font> via NNets

<font color="Coral"> _Regression_ </font> - estimating a floating-point value - is NNets' natural operation.

<br>

_Classification_ - estimating a discrete label - is done by creating discrete variables (we can't let the library treat a categorical variable as if it were continuous). <br> We'll start there, since it's a variation of our past workflows:


In [ ]:
#
# hw7pr1iris_modeler:  NNETS! 
#
#   including _both_ clasification + regression for iris modeling
#

In [1]:
# libraries...
import numpy as np      # numpy is Python's "array" library
import pandas as pd     # Pandas is Python's "data" library ("dataframe" == spreadsheet)

### Reading in our data

previously cleaned...

In [2]:
# let's read in our flower data...
# 
# iris_cleaned.csv should be in this folder
# 
filename = 'iris_cleaned.csv' # neighborhoods
df_tidy = pd.read_csv(filename)      # encoding = "utf-8", "latin1"
print(f"{filename} : file read into a pandas dataframe.")

iris_cleaned.csv : file read into a pandas dataframe.


In [3]:
#
# different version vary on how to see all rows (adapt to suit your system!)
#
print(f"df_tidy.shape is {df_tidy.shape}\n")
df_tidy.info()  # prints column information
df_tidy

df_tidy.shape is (141, 6)

<class 'pandas.core.frame.DataFrame'>
Index: 141 entries, 0 to 140
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   sepallen  141 non-null    float64
 1   sepalwid  141 non-null    float64
 2   petallen  141 non-null    float64
 3   petalwid  141 non-null    float64
 4   irisname  141 non-null    object 
 5   irisnum   141 non-null    int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 7.7+ KB


,sepallen,sepalwid,petallen,petalwid,irisname,irisnum
0,4.6,3.6,1.0,0.2,setosa,0
1,4.3,3.0,1.1,0.1,setosa,0
2,5.0,3.2,1.2,0.2,setosa,0
3,5.8,4.0,1.2,0.2,setosa,0
4,4.4,3.0,1.3,0.2,setosa,0
...,...,...,...,...,...,...
136,7.9,3.8,6.4,2.0,virginica,2
137,7.6,3.0,6.6,2.1,virginica,2
138,7.7,3.8,6.7,2.2,virginica,2
139,7.7,2.8,6.7,2.0,virginica,2


In [4]:
#
# we need to make sure the categories are handed as categories!
#    scikit-learn calls these "dummy" variables:

df_tidy_cat = pd.get_dummies(data=df_tidy,prefix="is",columns=['irisname'])
df_tidy_cat

,sepallen,sepalwid,petallen,petalwid,irisnum,is_setosa,is_versicolor,is_virginica
0,4.6,3.6,1.0,0.2,0,True,False,False
1,4.3,3.0,1.1,0.1,0,True,False,False
2,5.0,3.2,1.2,0.2,0,True,False,False
3,5.8,4.0,1.2,0.2,0,True,False,False
4,4.4,3.0,1.3,0.2,0,True,False,False
...,...,...,...,...,...,...,...,...
136,7.9,3.8,6.4,2.0,2,False,False,True
137,7.6,3.0,6.6,2.1,2,False,False,True
138,7.7,3.8,6.7,2.2,2,False,False,True
139,7.7,2.8,6.7,2.0,2,False,False,True


In [5]:
#
# We've removed the irisname column, and with our "" 
# we don't need the irisnum of the columns need to be numeric, we'll drop irisname
ROW = 0
COLUMN = 1
df_model1 = df_tidy_cat.drop('irisnum', axis=COLUMN )
df_model1

,sepallen,sepalwid,petallen,petalwid,is_setosa,is_versicolor,is_virginica
0,4.6,3.6,1.0,0.2,True,False,False
1,4.3,3.0,1.1,0.1,True,False,False
2,5.0,3.2,1.2,0.2,True,False,False
3,5.8,4.0,1.2,0.2,True,False,False
4,4.4,3.0,1.3,0.2,True,False,False
...,...,...,...,...,...,...,...
136,7.9,3.8,6.4,2.0,False,False,True
137,7.6,3.0,6.6,2.1,False,False,True
138,7.7,3.8,6.7,2.2,False,False,True
139,7.7,2.8,6.7,2.0,False,False,True


In [6]:
#
# once we have all the columns we want, let's create an index of their names...

#
# Let's make sure we have all of our helpful variables in one place 
#       To be adapted if we drop/add more columns...
#

#
# let's keep our column names in variables, for reference
#
COLUMNS = df_model1.columns            # "list" of columns
print(f"COLUMNS is {COLUMNS}\n")  
  # It's a "pandas" list, called an Index
  # use it just as a Python list of strings:
print(f"COLUMNS[0] is {COLUMNS[0]}\n")

# let's create a dictionary to look up any column index by name
COL_INDEX = {}
for i, name in enumerate(COLUMNS):
    COL_INDEX[name] = i  # using the name (as key), look up the value (i)
print(f"COL_INDEX is {COL_INDEX}\n\n")


#
# and our "species" names
#

# all of scikit-learn's ML routines need numbers, not strings
#   ... even for categories/classifications (like species!)
#   so, we will convert the flower-species to numbers:

SPECIES = ['setosa','versicolor','virginica']   # int to str
SPECIES_INDEX = {'setosa':0,'versicolor':1,'virginica':2}  # str to int

# Let's try it out...
for name in SPECIES:
    print(f"{name} maps to {SPECIES_INDEX[name]}")

COLUMNS is Index(['sepallen', 'sepalwid', 'petallen', 'petalwid', 'is_setosa',
       'is_versicolor', 'is_virginica'],
      dtype='object')

COLUMNS[0] is sepallen

COL_INDEX is {'sepallen': 0, 'sepalwid': 1, 'petallen': 2, 'petalwid': 3, 'is_setosa': 4, 'is_versicolor': 5, 'is_virginica': 6}


setosa maps to 0
versicolor maps to 1
virginica maps to 2


In [7]:
#
# We _could_ reweight our columns...  DON'T DO THIS!
# For example, if petalwid were "worth" 20x more than the others?
# 

# df_model1['petalwid'] *= 20
# df_model1

#
# But, with NNets, the whole goal of the network is to _adaptively_ weight each feature...
#
#      That's really all the network is doing... !?!!
#      Let's see it in action:
#

In [8]:
#
# let's convert our dataframe to a numpy array, named A
#
A = df_model1.to_numpy()   
A = A.astype('float64')    # many types:  www.tutorialspoint.com/numpy/numpy_data_types.htm
print(A[:,:])

# print(A)  # the five rows above is probably enough...   

[[4.6 3.6 1.  0.2 1.  0.  0. ]
 [4.3 3.  1.1 0.1 1.  0.  0. ]
 [5.  3.2 1.2 0.2 1.  0.  0. ]
 [5.8 4.  1.2 0.2 1.  0.  0. ]
 [4.4 3.  1.3 0.2 1.  0.  0. ]
 [4.4 3.2 1.3 0.2 1.  0.  0. ]
 [4.5 2.3 1.3 0.3 1.  0.  0. ]
 [4.7 3.2 1.3 0.2 1.  0.  0. ]
 [5.  3.5 1.3 0.3 1.  0.  0. ]
 [5.4 3.9 1.3 0.4 1.  0.  0. ]
 [5.5 3.5 1.3 0.2 1.  0.  0. ]
 [4.4 2.9 1.4 0.2 1.  0.  0. ]
 [4.6 3.4 1.4 0.3 1.  0.  0. ]
 [4.6 3.2 1.4 0.2 1.  0.  0. ]
 [4.8 3.  1.4 0.1 1.  0.  0. ]
 [4.8 3.  1.4 0.3 1.  0.  0. ]
 [4.9 3.  1.4 0.2 1.  0.  0. ]
 [5.  3.6 1.4 0.2 1.  0.  0. ]
 [5.  3.3 1.4 0.2 1.  0.  0. ]
 [5.1 3.5 1.4 0.2 1.  0.  0. ]
 [5.1 3.5 1.4 0.3 1.  0.  0. ]
 [5.2 3.4 1.4 0.2 1.  0.  0. ]
 [5.5 4.2 1.4 0.2 1.  0.  0. ]
 [4.6 3.1 1.5 0.2 1.  0.  0. ]
 [4.9 3.1 1.5 0.1 1.  0.  0. ]
 [4.9 3.1 1.5 0.1 1.  0.  0. ]
 [4.9 3.1 1.5 0.1 1.  0.  0. ]
 [5.  3.4 1.5 0.2 1.  0.  0. ]
 [5.1 3.8 1.5 0.3 1.  0.  0. ]
 [5.1 3.7 1.5 0.4 1.  0.  0. ]
 [5.1 3.4 1.5 0.2 1.  0.  0. ]
 [5.2 3.5 1.5 0.2 1.  0.  0. ]
 [5.3 3.

In [9]:
#
# nice to have NUM_ROWS and NUM_COLS around
#
NUM_ROWS, NUM_COLS = A.shape
print(f"\nThe dataset has {NUM_ROWS} rows and {NUM_COLS} cols")


The dataset has 141 rows and 7 cols


### We have full access to individual data elements

Checking on our observations and attributes

In [10]:
# let's use all of our variables, to reinforce that we have
# (1) names...
# (2) access and control...

# choose a row index, n:
n = 42
print(f"flower #{n} is {A[n]}")
species = ""

for i in range(len(COLUMNS)):
    colname = COLUMNS[i]
    value = A[n][i]
    print(f"  Its {colname} is {value}")
    if "setosa" in colname: species = "setosa"
    elif "versicolor" in colname: species = "versicolor"
    elif "virginica" in colname: species = "virginica"

print()
print(f"  and its species name: {species}")

flower #42 is [5.4 3.9 1.7 0.4 1.  0.  0. ]
  Its sepallen is 5.4
  Its sepalwid is 3.9
  Its petallen is 1.7
  Its petalwid is 0.4
  Its is_setosa is 1.0
  Its is_versicolor is 0.0
  Its is_virginica is 0.0

  and its species name: virginica


### Defining X_all (our features) and y_all (our target to predict) 

In [11]:
print("+++ Start of data definitions +++\n")

#
# we could do this at the data-frame level, too!
#

X_all = A[:,0:4]   # X (features) ... is all rows, columns 0, 1, 2, 3
y_all = A[:,4:]    # y (labels) ... is all rows, columns 4, 5, and 6

print(f"y_all (just the labels/species, first few rows) are \n {y_all[0:5]}")
print()
print(f"X_all (just the features, first few rows) are \n {X_all[0:5]}")

+++ Start of data definitions +++

y_all (just the labels/species, first few rows) are 
 [[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]

X_all (just the features, first few rows) are 
 [[4.6 3.6 1.  0.2]
 [4.3 3.  1.1 0.1]
 [5.  3.2 1.2 0.2]
 [5.8 4.  1.2 0.2]
 [4.4 3.  1.3 0.2]]


### Permute...

In [ ]:
#
# we can scramble the data, to remove (potential) dependence on its ordering: 
# 
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_all = X_all[indices]              # we apply the _same_ permutation to each!
y_all = y_all[indices]              # again...
print(f"The scrambled labels/species are \n {y_all[0:5]}")
print()
print(f"The corresponding data rows are \n {X_all[0:5]}")

### Separate into training and testing
+ X_train, y_train will be used for model-building
+ X_test, y_test will be used for testing, later

In [12]:
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

#
# a common convention:  train on 80%, test on 20%    Let's define the TEST_PERCENT
#

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2)

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

print(f"+++ Testing +++   Held-out data... (testing data: {len(y_test)})\n")
print(f"y_test: {y_test[0:5,:]}\n")
print(f"X_test (few rows): {X_test[0:5,:]}")  # 5 rows
print("\n")
print(f"+++ Training +++   Data used for modeling... (training data: {len(y_train)})\n")
print(f"y_train: {y_train[0:5,:]}\n")
print(f"X_train (few rows): {X_train[0:5,:]}")  # 5 rows

training with 112 rows;  testing with 29 rows

+++ Testing +++   Held-out data... (testing data: 29)

y_test: [[1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]]

X_test (few rows): [[5.1 3.8 1.9 0.4]
 [6.9 3.1 5.4 2.1]
 [6.4 2.7 5.3 1.9]
 [7.6 3.  6.6 2.1]
 [5.9 3.2 4.8 1.8]]


+++ Training +++   Data used for modeling... (training data: 112)

y_train: [[0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]

X_train (few rows): [[6.5 3.  5.8 2.2]
 [5.3 3.7 1.5 0.2]
 [4.8 3.  1.4 0.1]
 [5.7 3.8 1.7 0.3]
 [6.  3.  4.8 1.8]]


### NNets often need a ***Scaler***

This scales all of the data so that it's of a similar size, e.g., -1 to 1

Notice that this cell produces
+ X_train_scaled &nbsp; (the scaled training data)
   + y_train_scaled is the same as y_train &nbsp; (no scaling needed for the target)
+ X_test_scaled &nbsp; (the scaled testing data)
   + y_test_scaled is the same as y_test &nbsp; (no scaling needed for the target)

After this, we just use the scaled data...

In [13]:
#
# for NNets, it's important to keep the feature values near 0, say -1. to 1. or so
#    This is done through the "StandardScaler" in scikit-learn
# 
from sklearn.preprocessing import StandardScaler

#
# do we want to use a Scaler?
#
USE_SCALER = True   # this variable is important! It tracks if we need to use the scaler...

# we "train the scaler"  (computes the mean and standard deviation)
if USE_SCALER == True:
    scaler = StandardScaler()
    scaler.fit(X_train)  # Scale with the training data! ave becomes 0; stdev becomes 1
else:
    # this one does no scaling!  We still create it to be consistent:
    scaler = StandardScaler(copy=True, with_mean=False, with_std=False) # no scaling
    scaler.fit(X_train)  # still need to fit, though it does not change...

scaler   # is now defined and ready to use...

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++

# Here are our scaled training and testing sets:

X_train_scaled = scaler.transform(X_train) # scale!
X_test_scaled = scaler.transform(X_test) # scale!

y_train_scaled = y_train.copy()  # the predicted/desired labels are not scaled
y_test_scaled = y_test.copy()  # not using the scaler



### Intuition-building for the Scaled data

In [14]:
#
# let's create a table for showing our data and its predictions...
#
def ascii_table(X,y,scaler_to_invert=None):
    """ print a table of inputs and outputs """
    np.set_printoptions(precision=2)  # Let's use less precision
    if scaler_to_invert == None:  # don't use the scaler
        X = X
    else:
        X = scaler_to_invert.inverse_transform(X)
    print(f"{'input ':>58s} -> {'pred':^7s} {'des':<5s}") 
    for i in range(len(y)):
        # whoa! serious f-string formatting:
        print(f"{str(X[i,0:4]):>58s} -> {'?':^7s} {str(y[i]):<21s}")   # !s is str ...
    print()
    
# to show the table with the scaled data:
ascii_table(X_train_scaled[0:5,:],y_train_scaled[0:5],None)

# to show the table with the original data:
ascii_table(X_train_scaled[0:5,:],y_train_scaled[0:5],scaler_to_invert=scaler)

                                                    input  ->  pred   des  
                                 [ 0.77 -0.12  1.15  1.29] ->    ?    [0. 0. 1.]           
                                 [-0.65  1.54 -1.26 -1.28] ->    ?    [1. 0. 0.]           
                                 [-1.25 -0.12 -1.32 -1.41] ->    ?    [1. 0. 0.]           
                                 [-0.18  1.78 -1.15 -1.15] ->    ?    [1. 0. 0.]           
                                 [ 0.18 -0.12  0.59  0.78] ->    ?    [0. 0. 1.]           

                                                    input  ->  pred   des  
                                         [6.5 3.  5.8 2.2] ->    ?    [0. 0. 1.]           
                                         [5.3 3.7 1.5 0.2] ->    ?    [1. 0. 0.]           
                                         [4.8 3.  1.4 0.1] ->    ?    [1. 0. 0.]           
                                         [5.7 3.8 1.7 0.3] ->    ?    [1. 0. 0.]           
                   

### Training ML models is no problem

that is, if the library does it for us! &nbsp;&nbsp; 😃 🧶 ⛄️ 🌊 🪺 🦔 :-)

In [15]:
from sklearn.neural_network import MLPClassifier

#
# Here's where you can change the number of hidden layers
# and number of neurons!  It's in the tuple  hidden_layer_sizes:
#
nn_classifier = MLPClassifier(hidden_layer_sizes=(6,7),  
                    # hidden_layer_sizes=(6,7)   means   4 inputs -> 6 hidden -> 7 hidden -> 3 outputs
                    max_iter=500,      # how many times to train
                    # activation="tanh", # the "activation function" input -> output
                    # solver='sgd',      # the algorithm for optimizing weights
                    verbose=True,      # False to "mute" the training
                    shuffle=True,      # reshuffle the training epochs?
                    random_state=None, # set for reproduceability
                    learning_rate_init=.1,       # learning rate: the amt of error to backpropagate!
                    learning_rate = 'adaptive')  # soften feedback as it converges

# documentation:
# scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html 
#     Try other network sizes / other parameters ...

print("\n\n++++++++++  TRAINING:  begin  +++++++++++++++\n\n")
nn_classifier.fit(X_train_scaled, y_train_scaled)
print("\n++++++++++  TRAINING:   end  +++++++++++++++")
print(f"The analog prediction error (the loss) is {nn_classifier.loss_}")



++++++++++  TRAINING:  begin  +++++++++++++++


Iteration 1, loss = 1.91363117
Iteration 2, loss = 1.53693616
Iteration 3, loss = 1.11588962
Iteration 4, loss = 0.78456361
Iteration 5, loss = 0.54620768
Iteration 6, loss = 0.42415287
Iteration 7, loss = 0.30225577
Iteration 8, loss = 0.25100254
Iteration 9, loss = 0.20001210
Iteration 10, loss = 0.16704821
Iteration 11, loss = 0.12626981
Iteration 12, loss = 0.10334127
Iteration 13, loss = 0.09899549
Iteration 14, loss = 0.09269139
Iteration 15, loss = 0.09330402
Iteration 16, loss = 0.08585741
Iteration 17, loss = 0.08313818
Iteration 18, loss = 0.08448287
Iteration 19, loss = 0.08350148
Iteration 20, loss = 0.08437819
Iteration 21, loss = 0.08354011
Iteration 22, loss = 0.08046313
Iteration 23, loss = 0.07932059
Iteration 24, loss = 0.07856843
Iteration 25, loss = 0.07728739
Iteration 26, loss = 0.07783479
Iteration 27, loss = 0.07831588
Iteration 28, loss = 0.07738808
Iteration 29, loss = 0.07780129
Iteration 30, loss = 0.07661145

### Now, let's test!

In [16]:
#
# how did it do on the testing data?
#

def get_species(A):
    """ returns the species for A ~ [1 0 0] or [0 1 0] or ... """
    for i in range(len(SPECIES)):
        if A[i] == 1: 
            return SPECIES[i]  # note that this "takes the first one"
    return "no species" 

SEE_PROBS = False

def ascii_table_for_classifier(Xsc,y,nn,scaler):
    """ a table including predictions using nn.predict """
    predictions = nn.predict(Xsc)            # all predictions
    prediction_probs = nn.predict_proba(Xsc) # all prediction probabilities
    Xpr = scaler.inverse_transform(Xsc)      # Xpr is the "X to print": the unscaled data
    # count correct
    num_correct = 0
    # printing
    print(f"{'input ':>28s} -> {'pred':^12s} {'des.':^12s}") 
    for i in range(len(y)):
        pred = predictions[i]
        pred_probs = str(prediction_probs[i,:])
        desired = y[i].astype(int)
        # print(pred, desired, pred_probs)
        pred_species = get_species(pred)
        des_species  = get_species(desired)
        if pred_species != des_species: result = "  incorrect: " + pred_probs
        else: result = "  correct" + (": "+pred_probs if SEE_PROBS else "") ; num_correct += 1
        # Xpr = Xsc  # if you want to see the scaled versions
        print(f"{Xpr[i,0:4]!s:>28s} -> {pred_species:^12s} {des_species:12s} {result:^10s}") 
    print(f"\ncorrect predictions: {num_correct} out of {len(y)}")
    
#
# let's see how it did on the test data (also the training data!)
#
ascii_table_for_classifier(X_test_scaled,
                           y_test_scaled,
                           nn_classifier,
                           scaler)   


                      input  ->     pred         des.    
           [5.1 3.8 1.9 0.4] ->    setosa    setosa         correct 
           [6.9 3.1 5.4 2.1] ->  virginica   virginica      correct 
           [6.4 2.7 5.3 1.9] ->  virginica   virginica      correct 
           [7.6 3.  6.6 2.1] ->  virginica   virginica      correct 
           [5.9 3.2 4.8 1.8] ->  versicolor  versicolor     correct 
           [5.6 2.9 3.6 1.3] ->  versicolor  versicolor     correct 
           [5.  3.4 1.5 0.2] ->    setosa    setosa         correct 
           [6.7 3.1 4.7 1.5] ->  versicolor  versicolor     correct 
           [6.7 3.  5.  1.7] ->  virginica   versicolor     incorrect: [1.15e-14 1.81e-01 8.20e-01]
           [6.1 2.9 4.7 1.4] ->  versicolor  versicolor     correct 
           [7.7 3.  6.1 2.3] ->  virginica   virginica      correct 
           [6.7 2.5 5.8 1.8] ->  virginica   virginica      correct 
           [5.8 2.6 4.  1.2] ->  versicolor  versicolor     correct 
           [5.

### Let's see the network's weights...

In [17]:
#
# We don't usually look inside the NNet, but we can: it's open-box modeling...
#
if True:  # do we want to see all of the parameters?
    np.set_printoptions(precision=2)  # Let's use less precision
    nn = nn_classifier  # less to type?
    print("\n\n+++++ parameters, weights, etc. +++++\n")
    print(f"\nweights/coefficients:\n")
    for i, wts in enumerate(nn.coefs_):
        print(f"[[ Layer {i} ]]\n   has shape = {wts.shape} and weights =\n{wts}")
        print(f"   with intercepts:\n {nn.intercepts_[i]}\n")
    print()
    print(f"\nall parameters: {nn.get_params()}")



+++++ parameters, weights, etc. +++++


weights/coefficients:

[[ Layer 0 ]]
   has shape = (4, 6) and weights =
[[ 0.98  0.14 -1.73 -1.29 -0.21 -1.32]
 [-1.39 -1.24  1.1   1.35 -0.31  2.01]
 [ 0.23  0.36 -1.68 -1.7   2.21 -0.61]
 [ 1.47 -0.59 -1.37 -0.9   1.98 -0.58]]
   with intercepts:
 [ 1.94  1.19  0.16  1.03 -0.51  1.28]

[[ Layer 1 ]]
   has shape = (6, 7) and weights =
[[ 1.12  0.43 -0.68  0.69  1.58 -1.35  0.27]
 [ 1.45 -0.18 -0.34 -0.75  0.99 -0.48  0.32]
 [-2.45 -0.33  0.07 -0.51 -1.02  1.37 -0.13]
 [ 0.05 -0.35 -0.71 -0.92  0.65  1.69 -0.34]
 [ 0.14  1.14 -0.63  1.54 -1.56 -1.86  1.79]
 [-0.17 -0.67 -0.6   0.4   1.3   1.52 -2.09]]
   with intercepts:
 [ 0.53 -0.44  0.02 -1.02  0.85  0.44 -1.22]

[[ Layer 2 ]]
   has shape = (7, 3) and weights =
[[-1.89  0.03 -0.17]
 [ 0.35 -0.75  0.05]
 [ 1.35 -0.45 -0.97]
 [-1.44 -0.35  0.8 ]
 [-1.73  1.56 -1.28]
 [ 1.97 -1.57 -1.91]
 [-0.62 -1.51  1.8 ]]
   with intercepts:
 [-1.06  0.2  -1.26]



all parameters: {'activation': 'relu', 

### The predictive model

In [18]:
#
# final predictive model (random forests), with tuned parameters + ALL data incorporated
#

def get_species(A):
    """ returns the species for A ~ [1 0 0] or [0 1 0] or ... """
    for i in range(len(SPECIES)):
        if A[i] == 1: 
            return SPECIES[i]  # note that this "takes the first one"
    return "no species" 


def predictive_model( Features, MODEL, SCALER ):
    """ input: a list of four features 
                [ sepallen, sepalwid, petallen, petalwid ]
        output: the predicted species of iris, from
                  setosa (0), versicolor (1), virginica (2)
    """
    our_features = np.asarray([Features])                 # extra brackets needed
    scaled_features = SCALER.transform(our_features)      # we have to scale the features into "scaled space"
    predicted_cat = MODEL.predict(scaled_features)        # then, the nnet can predict our "cat" variables
    prediction_probs = nn.predict_proba(scaled_features) # all prediction probabilities
    # our_features = SCALER.inverse_transform(scaled_features)  # we can convert back (optional!)
    predicted_species = get_species(predicted_cat[0])     # (it's extra-nested) get the species name
    return predicted_species, prediction_probs
   
#
# Try it!
# 
# Features = eval(input("Enter new Features: "))
#
Features = [6.7,3.3,5.7,0.1]  # [5.8,2.7,4.1,1.0] [4.6,3.6,3.0,2.2] [6.7,3.3,5.7,2.1]

LoF = [
[4.8, 3.1, 1.6, 0.2 ],   # actually setosa
[5.7, 2.9, 4.2, 1.3 ],   # actually versicolor
[5.8, 2.7, 5.1, 1.9 ],   # actually virginica
[5.2, 4.1, 1.5, 0.1 ],   # actually setosa
[5.4, 3.4, 1.5, 0.4 ],   # actually setosa
[5.1, 2.5, 3.0, 1.1 ],   # actually versicolor
[6.2, 2.9, 4.3, 1.3 ],   # actually versicolor
[6.3, 3.3, 6.0, 2.5 ],   # actually virginica
[5.7, 2.8, 4.1, 1.3 ],   # actually virginica  <-- almost always wrong!
]

SEE_PROBS = False

# run on each one:
for Features in LoF:
    MODEL = nn_classifier
    SCALER = scaler
    name, probs = predictive_model( Features, MODEL, SCALER )  # pass in the model, too!
    prob_str = "   with probs: " + str(probs) if SEE_PROBS == True else ""
    print(f"I predict {name:>12s} from the features {Features}  {prob_str}")    # Answers in the assignment...

I predict       setosa from the features [4.8, 3.1, 1.6, 0.2]  
I predict   versicolor from the features [5.7, 2.9, 4.2, 1.3]  
I predict    virginica from the features [5.8, 2.7, 5.1, 1.9]  
I predict       setosa from the features [5.2, 4.1, 1.5, 0.1]  
I predict       setosa from the features [5.4, 3.4, 1.5, 0.4]  
I predict   versicolor from the features [5.1, 2.5, 3.0, 1.1]  
I predict   versicolor from the features [6.2, 2.9, 4.3, 1.3]  
I predict    virginica from the features [6.3, 3.3, 6.0, 2.5]  
I predict   versicolor from the features [5.7, 2.8, 4.1, 1.3]  


<br>

#### We _could_ use cross-validation to find the "right"/"best" size and shape of the NNet...

<font color="Coral">But, let's not...</font>



### <font color="DodgerBlue">Instead, we'll explore two new directions:</font> 
+ regression and 
+ feature-predictability

#### The next example...
+ is one in which we estimate a botanical feature ... 
+ ... from the other four columns -- including the species!

In [19]:
#
# What shall we predict today?
print(COL_INDEX)
print()
print(COLUMNS)

# Let's first predict sepal length ('sepallen', column index 0)

{'sepallen': 0, 'sepalwid': 1, 'petallen': 2, 'petalwid': 3, 'is_setosa': 4, 'is_versicolor': 5, 'is_virginica': 6}

Index(['sepallen', 'sepalwid', 'petallen', 'petalwid', 'is_setosa',
       'is_versicolor', 'is_virginica'],
      dtype='object')


In [20]:
#
# Here we set up for a regression model that will predict 'sepallen'  (column index 0)
#
#   sepal length  'sepallen' (column index 0)  <-- our target for _regression_

# We will use
#
#   sepal width   'sepalwid' (column index 1)
#   petal length  'petallen' (column index 2)
#   petal width   'petalwid' (column index 3)
#   _and_
#   species       'irisnum'  (column index 4)    # No problem - we can use this as a feature!

print("+++ Start of data-assembly for feature-regression! +++\n")
# construct the correct X_all from the columns we want
# we use np.concatenate to combine parts of the dataset to get all-except-column 0:
#                     exclude 0  , include 1 to the end
X_all = np.concatenate( (A[:,0:0], A[:,1:]), axis=1)    # includes columns 1, 2, 3, and 4

# if we wanted all-except-column 1:   X_all = np.concatenate( (A[:,0:1], A[:,2:]),axis=1)  # columns 0, 2, 3, and 4
# if we wanted all-except-column 2:   X_all = np.concatenate( (A[:,0:2], A[:,3:]),axis=1)  # columns 0, 1, 3, and 4
# if we wanted all-except-column 3:   X_all = np.concatenate( (A[:,0:3], A[:,4:]),axis=1)  # columns 0, 1, 2, and 4
# if we wanted all-except-column 4:   X_all = np.concatenate( (A[:,0:4], A[:,5:]),axis=1)  # columns 0, 1, 2, and 3
# (slicing is forgiving)


y_all = A[:,0]                    # y (labels) ... is all of column 0, sepallen (sepal length) 
#                                 # change the line above to make other columns the target (y_all)
print(f"y_all is \n {y_all}")
print() 
print(f"X_all (the features, first few rows) is \n {X_all[:5,:]}")


+++ Start of data-assembly for feature-regression! +++

y_all is 
 [4.6 4.3 5.  5.8 4.4 4.4 4.5 4.7 5.  5.4 5.5 4.4 4.6 4.6 4.8 4.8 4.9 5.
 5.  5.1 5.1 5.2 5.5 4.6 4.9 4.9 4.9 5.  5.1 5.1 5.1 5.2 5.3 5.4 5.7 4.7
 4.8 5.  5.  5.  5.1 5.1 5.4 5.4 5.7 4.8 5.1 7.  4.9 5.  5.  5.7 5.6 5.5
 5.5 5.2 5.6 5.8 5.5 5.5 5.8 6.  6.1 5.6 5.8 5.6 5.7 5.9 6.4 5.5 6.3 6.6
 6.7 5.4 5.6 5.7 6.  6.  6.2 6.4 6.1 6.5 6.6 6.1 6.1 6.3 6.7 5.9 6.8 6.3
 6.9 6.7 6.  4.9 6.  6.2 5.6 6.1 6.3 5.7 6.  6.3 5.8 5.8 5.9 6.3 6.5 6.9
 6.5 6.7 6.4 6.4 6.2 6.9 6.4 6.5 6.8 6.1 6.3 6.3 6.4 6.4 6.7 6.7 6.7 6.9
 6.5 6.7 7.2 6.8 7.1 7.2 7.2 7.4 7.7 7.3 7.9 7.6 7.7 7.7 7.7]

X_all (the features, first few rows) is 
 [[3.6 1.  0.2 1.  0.  0. ]
 [3.  1.1 0.1 1.  0.  0. ]
 [3.2 1.2 0.2 1.  0.  0. ]
 [4.  1.2 0.2 1.  0.  0. ]
 [3.  1.3 0.2 1.  0.  0. ]]


### Permute!

<font size="-2">I like this because it adds an element of randomness to the whole process...</font>

In [21]:
#
# we scramble the data, to give a different TRAIN/TEST split each time...
# 
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_all = X_all[indices]              # we apply the _same_ permutation to each!
y_all = y_all[indices]              # again...
print("target values to predict: \n",y_all)
print("\nfeatures (a few)\n", X_all[0:5,:])

target values to predict: 
 [5.  6.7 6.7 4.9 5.9 6.3 6.4 7.2 5.4 6.4 4.4 6.5 4.6 5.7 6.  6.1 5.6 6.3
 5.8 6.4 5.7 4.9 6.7 4.3 5.  6.3 7.7 5.8 6.5 6.9 6.5 6.7 6.3 5.7 5.1 5.7
 4.8 5.  4.7 6.3 7.3 5.1 6.4 5.5 6.4 4.4 5.  5.5 5.5 6.  6.5 6.8 6.4 6.7
 6.2 4.6 6.5 6.  6.  7.7 5.7 5.6 5.8 4.8 5.4 6.1 5.  5.1 5.  4.5 4.8 5.1
 6.  6.1 4.9 5.1 7.  7.6 6.4 5.6 7.2 6.9 7.2 4.9 5.1 6.9 5.6 5.  5.1 6.8
 6.6 6.3 6.3 6.  5.8 6.2 6.9 4.8 4.9 5.2 7.9 5.  6.1 4.6 6.7 5.9 4.6 6.8
 4.4 5.6 5.4 5.  5.7 5.1 5.  5.8 6.6 4.7 7.4 6.7 7.7 5.6 7.1 6.2 5.5 5.2
 6.1 5.2 6.3 5.9 7.7 5.4 5.5 5.4 5.5 4.9 6.7 5.8 5.5 5.3 6.1]

features (a few)
 [[3.5 1.6 0.6 1.  0.  0. ]
 [3.1 4.4 1.4 0.  1.  0. ]
 [3.3 5.7 2.5 0.  0.  1. ]
 [3.1 1.5 0.1 1.  0.  0. ]
 [3.  4.2 1.5 0.  1.  0. ]]


### Split into training and testing sets

We'll stick with 80% training, 20% testing...

In [22]:
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2)

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

print(f"Held-out data... (testing data: {len(y_test)})")
print(f"y_test: {y_test}\n")
print(f"X_test (few rows): {X_test[0:5,:]}")  # 5 rows
print()
print(f"Data used for modeling... (training data: {len(y_train)})")
print(f"y_train: {y_train}\n")
print(f"X_train (few rows): {X_train[0:5,:]}")  # 5 rows

training with 112 rows;  testing with 29 rows

Held-out data... (testing data: 29)
y_test: [6.7 6.7 7.2 5.7 6.7 5.6 6.3 5.  6.4 4.8 7.9 6.5 7.7 6.3 6.4 6.2 5.4 5.6
 7.7 5.5 6.5 5.1 6.  5.  6.4 5.9 5.8 5.  5.6]

X_test (few rows): [[3.1 4.4 1.4 0.  1.  0. ]
 [3.1 4.7 1.5 0.  1.  0. ]
 [3.  5.8 1.6 0.  0.  1. ]
 [4.4 1.5 0.4 1.  0.  0. ]
 [3.  5.2 2.3 0.  0.  1. ]]

Data used for modeling... (training data: 112)
y_train: [5.  5.7 5.4 5.5 5.5 6.4 4.9 5.7 6.3 6.1 4.6 6.7 6.  6.6 4.9 7.7 4.4 6.1
 5.3 5.4 5.  4.9 5.8 4.8 5.  4.4 6.2 6.4 5.7 5.4 4.8 6.9 6.  6.9 6.8 5.6
 5.1 4.5 5.7 6.  6.3 5.5 7.  5.5 6.8 5.1 6.5 5.1 6.7 5.8 7.6 6.3 6.3 4.6
 6.4 7.1 4.9 5.2 6.1 6.3 6.7 5.  4.9 5.1 6.7 5.6 5.8 5.2 4.3 5.2 6.6 6.2
 4.7 6.7 5.6 7.7 6.  5.  5.  5.8 6.3 5.1 5.1 5.8 5.5 5.  6.  5.7 7.2 5.5
 4.6 6.4 4.9 7.3 6.1 6.1 4.8 5.9 6.9 6.9 5.1 6.5 6.1 6.5 4.6 7.4 4.4 5.4
 6.8 7.2 5.9 4.7]

X_train (few rows): [[3.3 1.4 0.2 1.  0.  0. ]
 [3.  4.2 1.2 0.  1.  0. ]
 [3.9 1.3 0.4 1.  0.  0. ]
 [3.5 1.3 0.2 1.  0

### It's a nnet - this time, we _should_ use a <u>Scaler</u>

<font size="-2">Note that, for our <u>own</u> neural nets, this is performed by our senses:
+ our sense of hearing scales sounds from whispers to shouts... 
    + ... into a single amplitude for interpretation. 
    + and, we keep the scale itself as an additional signal!
+ similarly, our visual system scales across many orders of magnitude of brightness
    + plus, our eyes can "adapt" to low light (and, less, to bright-light )
+ less dramatically, for taste, touch, and smell...
+ this scaling 

In [23]:
#
# for NNets, it's important to keep the feature values near 0, say -1. to 1. or so
#    This is done through the "StandardScaler" in scikit-learn
#
from sklearn.preprocessing import StandardScaler

USE_SCALER = True   # this variable is important! It tracks if we need to use the scaler...

# we "train the scaler"  (computes the mean and standard deviation)
if USE_SCALER == True:
    scaler = StandardScaler()
    scaler.fit(X_train)  # Scale with the training data! ave becomes 0; stdev becomes 1
else:
    # this one does no scaling!  We still create it to be consistent:
    scaler = StandardScaler(copy=True, with_mean=False, with_std=False) # no scaling
    scaler.fit(X_train)  # still need to fit, though it does not change...

scaler   # is now defined and ready to use...

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++

# Here are our scaled training and testing sets:

X_train_scaled = scaler.transform(X_train) # scale!
X_test_scaled = scaler.transform(X_test) # scale!

y_train_scaled = y_train  # the predicted/desired labels are not scaled
y_test_scaled = y_test  # not using the scaler

# reused from above - seeing the scaled data 
ascii_table(X_train_scaled[0:5,:],y_train_scaled[0:5],None)

# reused from above - seeing the unscaled data (inverting the scaler)
ascii_table(X_train_scaled[0:5,:],y_train_scaled[0:5],scaler)

                                                    input  ->  pred   des  
                                 [ 0.6  -1.24 -1.22  1.29] ->    ?    5.0                  
                                 [-0.15  0.32  0.08 -0.77] ->    ?    5.7                  
                                 [ 2.08 -1.3  -0.96  1.29] ->    ?    5.4                  
                                 [ 1.09 -1.3  -1.22  1.29] ->    ?    5.5                  
                                 [-1.14  0.43  0.08 -0.77] ->    ?    5.5                  

                                                    input  ->  pred   des  
                                         [3.3 1.4 0.2 1. ] ->    ?    5.0                  
                                         [3.  4.2 1.2 0. ] ->    ?    5.7                  
                                         [3.9 1.3 0.4 1. ] ->    ?    5.4                  
                                         [3.5 1.3 0.2 1. ] ->    ?    5.5                  
                   

### Create and train the model

Note that hidden_layer_sizes is an important parameter.
+ for this hw, we're simply trying things out
+ no need to optimize via cross-validation

In [24]:
#
# MLPRegressor predicts _floating-point_ outputs
#

from sklearn.neural_network import MLPRegressor

nn_regressor = MLPRegressor(hidden_layer_sizes=(6,7), 
                    max_iter=500,          # how many training epochs
                    verbose=True,          # do we want to watch as it trains?
                    shuffle=True,          # shuffle each epoch?
                    random_state=None,     # use for reproducibility
                    learning_rate_init=.1, # how much of each error to back-propagate
                    learning_rate = 'adaptive')  # how to handle the learning_rate

print("\n\n++++++++++  TRAINING:  begin  +++++++++++++++\n\n")
nn_regressor.fit(X_train_scaled, y_train_scaled)
print("++++++++++  TRAINING:   end  +++++++++++++++")
print(f"The (squared) prediction error (the loss) is {nn_regressor.loss_:<6.3f}")
print(f"And, its square root:         {nn_regressor.loss_ ** 0.5:<6.3f}")
print()



++++++++++  TRAINING:  begin  +++++++++++++++


Iteration 1, loss = 20.24048963
Iteration 2, loss = 15.66295974
Iteration 3, loss = 10.12842413
Iteration 4, loss = 4.28191604
Iteration 5, loss = 4.05464989
Iteration 6, loss = 5.53629399
Iteration 7, loss = 2.98190897
Iteration 8, loss = 0.72174443
Iteration 9, loss = 0.48479392
Iteration 10, loss = 1.20509887
Iteration 11, loss = 1.84963414
Iteration 12, loss = 2.06814802
Iteration 13, loss = 1.88191829
Iteration 14, loss = 1.45192556
Iteration 15, loss = 0.94033384
Iteration 16, loss = 0.51141522
Iteration 17, loss = 0.31266387
Iteration 18, loss = 0.37263133
Iteration 19, loss = 0.56807104
Iteration 20, loss = 0.69000206
Iteration 21, loss = 0.64055994
Iteration 22, loss = 0.49979424
Iteration 23, loss = 0.38508995
Iteration 24, loss = 0.34185867
Iteration 25, loss = 0.33360214
Iteration 26, loss = 0.30809408
Iteration 27, loss = 0.24532660
Iteration 28, loss = 0.16789418
Iteration 29, loss = 0.11911039
Iteration 30, loss = 0.12909

In [ ]:
#
# Note that square-root of the mean-squared-error is an "expected error" in predicting our feature
#
# It's not the only estimate of expected error, but it's easier to think about than the squared error (which has the wrong units)
#

### Let's test!  

Testing is different for regression!  
<font size="-1">
+ it's floating-point, so it's no longer "right" or "wrong"
+ instead, the _difference_ is what matters
+ in fact, the <u> _absolute difference_ </u> is usually what matters: </font>

In [34]:
#
# how did it do? Try out the TEST data...
#

def ascii_table_for_regressor(Xsc,y,nn,scaler):
    """ a table including predictions using nn.predict """
    predictions = nn.predict(Xsc) # all predictions
    Xpr = scaler.inverse_transform(Xsc)  # Xpr is the "X to print": unscaled data!
    # measure error
    error = 0.0
    # printing
    print(f"{'input ':>35s} ->  {'pred':^6s}  {'des.':^6s}   {'absdiff':^10s}") 
    for i in range(len(y)):
        pred = predictions[i]
        desired = y[i]
        result = abs(desired - pred)
        error += result
        # Xpr = Xsc   # if you'd like to see the scaled values
        print(f"{Xpr[i,:]!s:>35s} ->  {pred:<+6.2f}  {desired:<+6.2f}   {result:^10.2f}") 

    print("\n" + "+++++   +++++   +++++           ")
    print(f"average abs diff error:   {error/len(y):<6.3f}")
    print("+++++   +++++   +++++           ")
    
#
# let's see how it did on the test data (also the training data!)
#
ascii_table_for_regressor(X_test_scaled,
                          y_test_scaled,
                          nn_regressor,
                          scaler)   # this is our own f'n, above


                             input  ->   pred    des.     absdiff  
          [5.4 1.3 0.4 1.  0.  0. ] ->  +3.74   +3.90       0.16   
          [4.7 1.6 0.2 1.  0.  0. ] ->  +3.11   +3.20       0.09   
          [4.3 1.1 0.1 1.  0.  0. ] ->  +3.04   +3.00       0.04   
          [6.3 4.4 1.3 0.  1.  0. ] ->  +2.88   +2.30       0.58   
          [4.6 1.5 0.2 1.  0.  0. ] ->  +3.08   +3.10       0.02   
          [5.5 4.  1.3 0.  1.  0. ] ->  +2.67   +2.50       0.17   
          [6.6 4.6 1.3 0.  1.  0. ] ->  +2.96   +2.90       0.06   
          [6.1 4.6 1.4 0.  1.  0. ] ->  +2.93   +3.00       0.07   
          [6.5 5.8 2.2 0.  0.  1. ] ->  +3.14   +3.00       0.14   
          [7.9 6.4 2.  0.  0.  1. ] ->  +3.09   +3.80       0.71   
          [6.8 5.5 2.1 0.  0.  1. ] ->  +2.99   +3.00       0.01   
          [5.8 3.9 1.2 0.  1.  0. ] ->  +2.62   +2.70       0.08   
          [7.1 5.9 2.1 0.  0.  1. ] ->  +3.07   +3.00       0.07   
          [5.7 3.5 1.  0.  1.  0. ] ->  +2.37   

### Your task:  &nbsp;&nbsp; model ***all*** of the iris features!

Which of the four floating-point iris features is
+ _most_ predictable? &nbsp; (smallest average abs diff error)
+ _least_ predictable? &nbsp; (largest average abs diff error)

<br>

As with the _feature importances_ , it's often the <u>insight</u> we value, more than creating the predictor itself...

## ***Sepalwid***

In [29]:
#
# Here we set up for a regression model that will predict 'sepallen'  (column index 0)
#
#   sepal length  'sepallen' (column index 0) 

# We will use
#
#   sepal width   'sepalwid' (column index 1) <-- our target for _regression_
#   petal length  'petallen' (column index 2)
#   petal width   'petalwid' (column index 3)
#   _and_
#   species       'irisnum'  (column index 4)    # No problem - we can use this as a feature!

print("+++ Start of data-assembly for feature-regression! +++\n")
# construct the correct X_all from the columns we want
# we use np.concatenate to combine parts of the dataset to get all-except-column 0:
#                     exclude 0  , include 1 to the end
X_all = np.concatenate( (A[:,0:1], A[:,2:]),axis=1)    # includes columns 1, 2, 3, and 4

# if we wanted all-except-column 1:   X_all = np.concatenate( (A[:,0:1], A[:,2:]),axis=1)  # columns 0, 2, 3, and 4
# if we wanted all-except-column 2:   X_all = np.concatenate( (A[:,0:2], A[:,3:]),axis=1)  # columns 0, 1, 3, and 4
# if we wanted all-except-column 3:   X_all = np.concatenate( (A[:,0:3], A[:,4:]),axis=1)  # columns 0, 1, 2, and 4
# if we wanted all-except-column 4:   X_all = np.concatenate( (A[:,0:4], A[:,5:]),axis=1)  # columns 0, 1, 2, and 3
# (slicing is forgiving)


y_all = A[:,1]                    # y (labels) ... is all of column 0, sepallen (sepal length) 
#                                 # change the line above to make other columns the target (y_all)
print(f"y_all is \n {y_all}")
print() 
print(f"X_all (the features, first few rows) is \n {X_all[:5,:]}")


+++ Start of data-assembly for feature-regression! +++

y_all is 
 [3.6 3.  3.2 4.  3.  3.2 2.3 3.2 3.5 3.9 3.5 2.9 3.4 3.2 3.  3.  3.  3.6
 3.3 3.5 3.5 3.4 4.2 3.1 3.1 3.1 3.1 3.4 3.8 3.7 3.4 3.5 3.7 3.7 4.4 3.2
 3.4 3.  3.4 3.5 3.8 3.3 3.9 3.4 3.8 3.4 3.8 3.2 2.4 2.3 2.  2.6 2.9 2.4
 2.4 2.7 2.5 2.7 2.3 2.5 2.6 2.2 2.8 3.  2.7 2.7 3.  3.  2.9 2.6 2.3 3.
 3.1 3.  3.  2.8 2.9 3.4 2.2 3.2 3.  2.8 2.9 2.9 2.8 3.3 3.1 3.2 2.8 2.5
 3.1 3.  2.7 2.5 3.  2.8 2.8 3.  2.7 2.5 2.2 2.5 2.8 2.7 3.  2.8 3.2 3.1
 3.  3.  2.7 3.2 3.4 3.1 3.1 3.  3.  2.6 2.9 3.4 2.8 2.8 3.1 3.3 3.3 3.2
 3.  2.5 3.  3.2 3.  3.2 3.6 2.8 3.  2.9 3.8 3.  3.8 2.8 2.6]

X_all (the features, first few rows) is 
 [[4.6 1.  0.2 1.  0.  0. ]
 [4.3 1.1 0.1 1.  0.  0. ]
 [5.  1.2 0.2 1.  0.  0. ]
 [5.8 1.2 0.2 1.  0.  0. ]
 [4.4 1.3 0.2 1.  0.  0. ]]


### Permute!

In [30]:
#
# we scramble the data, to give a different TRAIN/TEST split each time...
# 
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_all = X_all[indices]              # we apply the _same_ permutation to each!
y_all = y_all[indices]              # again...
print("target values to predict: \n",y_all)
print("\nfeatures (a few)\n", X_all[0:5,:])

target values to predict: 
 [2.5 3.6 2.8 2.7 3.1 2.9 2.7 3.7 2.5 2.6 3.1 3.4 3.  2.2 3.  3.8 3.1 3.3
 3.  3.4 3.2 2.7 3.1 3.8 3.7 2.3 2.6 3.  3.1 2.4 3.2 2.7 4.  3.5 3.  2.8
 2.4 2.9 2.9 3.  3.4 3.3 3.  3.5 2.8 2.  2.4 3.4 3.  3.2 2.9 3.5 3.  3.
 3.6 3.1 2.3 3.  2.8 3.  3.2 3.2 2.7 3.9 3.  3.1 3.  3.5 3.4 2.8 3.2 2.9
 2.3 2.6 2.8 3.4 3.5 3.2 2.9 2.6 3.2 3.6 2.6 3.2 3.4 3.8 3.3 4.2 2.8 3.1
 3.2 2.8 3.9 2.7 3.8 3.8 2.8 3.  3.  3.  3.  2.5 2.7 3.5 3.4 3.2 2.8 3.3
 2.5 2.2 3.  2.8 2.8 2.2 3.  4.4 3.8 3.2 3.1 3.1 3.  2.5 3.  2.9 3.4 3.
 3.  2.5 3.  2.9 3.3 3.2 2.3 3.  3.7 3.4 2.5 2.7 2.8 3.1 3.4]

features (a few)
 [[5.5 4.  1.3 0.  1.  0. ]
 [5.  1.4 0.2 1.  0.  0. ]
 [6.3 5.1 1.5 0.  0.  1. ]
 [5.2 3.9 1.4 0.  1.  0. ]
 [4.9 1.5 0.1 1.  0.  0. ]]


### Split into training and testing sets

We'll stick with 80% training, 20% testing...

In [31]:
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2)

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

print(f"Held-out data... (testing data: {len(y_test)})")
print(f"y_test: {y_test}\n")
print(f"X_test (few rows): {X_test[0:5,:]}")  # 5 rows
print()
print(f"Data used for modeling... (training data: {len(y_train)})")
print(f"y_train: {y_train}\n")
print(f"X_train (few rows): {X_train[0:5,:]}")  # 5 rows

training with 112 rows;  testing with 29 rows

Held-out data... (testing data: 29)
y_test: [3.9 3.2 3.  2.3 3.1 2.5 2.9 3.  3.  3.8 3.  2.7 3.  2.6 3.  3.2 3.4 2.9
 2.4 2.6 2.3 3.4 2.9 2.7 3.8 3.2 2.2 3.  2.8]

X_test (few rows): [[5.4 1.3 0.4 1.  0.  0. ]
 [4.7 1.6 0.2 1.  0.  0. ]
 [4.3 1.1 0.1 1.  0.  0. ]
 [6.3 4.4 1.3 0.  1.  0. ]
 [4.6 1.5 0.2 1.  0.  0. ]]

Data used for modeling... (training data: 112)
y_train: [3.4 3.1 3.1 2.8 2.8 3.  3.5 3.2 3.3 2.8 3.1 3.5 2.6 3.3 3.3 2.5 2.8 3.
 2.9 2.8 2.7 3.8 3.4 3.7 3.8 2.8 2.7 4.4 2.  3.  3.1 2.9 3.4 3.  2.8 3.8
 2.9 3.2 3.2 3.  3.4 3.  3.1 2.3 3.2 3.4 2.3 2.5 2.5 3.6 3.1 3.  2.8 3.5
 3.1 3.1 3.3 3.  4.  3.2 3.7 3.1 3.  3.  3.7 3.2 3.1 2.5 2.2 2.4 2.4 3.6
 3.4 3.  3.2 3.  3.8 3.  2.7 3.4 3.4 2.7 3.9 3.  2.6 3.2 3.4 2.5 3.  3.5
 3.5 3.2 2.9 2.8 3.  2.6 2.7 2.5 3.3 2.8 3.5 2.8 3.  4.2 3.  3.2 2.9 3.6
 2.8 2.2 2.7 3. ]

X_train (few rows): [[6.  4.5 1.6 0.  1.  0. ]
 [4.9 1.5 0.1 1.  0.  0. ]
 [6.9 4.9 1.5 0.  1.  0. ]
 [6.8 4.8 1.4 0.  1.

### It's a nnet - this time, we _should_ use a <u>Scaler</u>

In [36]:
#
# for NNets, it's important to keep the feature values near 0, say -1. to 1. or so
#    This is done through the "StandardScaler" in scikit-learn
#
from sklearn.preprocessing import StandardScaler

USE_SCALER = True   # this variable is important! It tracks if we need to use the scaler...

# we "train the scaler"  (computes the mean and standard deviation)
if USE_SCALER == True:
    scaler = StandardScaler()
    scaler.fit(X_train)  # Scale with the training data! ave becomes 0; stdev becomes 1
else:
    # this one does no scaling!  We still create it to be consistent:
    scaler = StandardScaler(copy=True, with_mean=False, with_std=False) # no scaling
    scaler.fit(X_train)  # still need to fit, though it does not change...

scaler   # is now defined and ready to use...

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++

# Here are our scaled training and testing sets:

X_train_scaled = scaler.transform(X_train) # scale!
X_test_scaled = scaler.transform(X_test) # scale!

y_train_scaled = y_train  # the predicted/desired labels are not scaled
y_test_scaled = y_test  # not using the scaler

# reused from above - seeing the scaled data 
ascii_table(X_train_scaled[0:5,:],y_train_scaled[0:5],None)

# reused from above - seeing the unscaled data (inverting the scaler)
ascii_table(X_train_scaled[0:5,:],y_train_scaled[0:5],scaler)

                                                    input  ->  pred   des  
                                 [ 0.19  0.44  0.52 -0.73] ->    ?    3.4                  
                                 [-1.15 -1.25 -1.42  1.37] ->    ?    3.1                  
                                 [ 1.3   0.66  0.39 -0.73] ->    ?    3.1                  
                                 [ 1.18  0.61  0.26 -0.73] ->    ?    2.8                  
                                 [ 0.32  0.16  0.13 -0.73] ->    ?    2.8                  

                                                    input  ->  pred   des  
                                         [6.  4.5 1.6 0. ] ->    ?    3.4                  
                                         [4.9 1.5 0.1 1. ] ->    ?    3.1                  
                                         [6.9 4.9 1.5 0. ] ->    ?    3.1                  
                                         [6.8 4.8 1.4 0. ] ->    ?    2.8                  
                   

### Create and train the model

In [37]:
#
# MLPRegressor predicts _floating-point_ outputs
#

from sklearn.neural_network import MLPRegressor

nn_regressor = MLPRegressor(hidden_layer_sizes=(6,7), 
                    max_iter=500,          # how many training epochs
                    verbose=True,          # do we want to watch as it trains?
                    shuffle=True,          # shuffle each epoch?
                    random_state=None,     # use for reproducibility
                    learning_rate_init=.1, # how much of each error to back-propagate
                    learning_rate = 'adaptive')  # how to handle the learning_rate

print("\n\n++++++++++  TRAINING:  begin  +++++++++++++++\n\n")
nn_regressor.fit(X_train_scaled, y_train_scaled)
print("++++++++++  TRAINING:   end  +++++++++++++++")
print(f"The (squared) prediction error (the loss) is {nn_regressor.loss_:<6.3f}")
print(f"And, its square root:         {nn_regressor.loss_ ** 0.5:<6.3f}")
print()



++++++++++  TRAINING:  begin  +++++++++++++++


Iteration 1, loss = 4.01874647
Iteration 2, loss = 2.55605093
Iteration 3, loss = 2.09531276
Iteration 4, loss = 1.21103634
Iteration 5, loss = 0.37384430
Iteration 6, loss = 0.12139659
Iteration 7, loss = 0.36121380
Iteration 8, loss = 0.27284288
Iteration 9, loss = 0.15063192
Iteration 10, loss = 0.10291398
Iteration 11, loss = 0.07534178
Iteration 12, loss = 0.09567039
Iteration 13, loss = 0.14360934
Iteration 14, loss = 0.14507377
Iteration 15, loss = 0.09802810
Iteration 16, loss = 0.06075922
Iteration 17, loss = 0.05360087
Iteration 18, loss = 0.06308527
Iteration 19, loss = 0.06846162
Iteration 20, loss = 0.06474995
Iteration 21, loss = 0.05872893
Iteration 22, loss = 0.05885977
Iteration 23, loss = 0.06163440
Iteration 24, loss = 0.05867836
Iteration 25, loss = 0.04994017
Iteration 26, loss = 0.04414638
Iteration 27, loss = 0.04520953
Iteration 28, loss = 0.04951274
Iteration 29, loss = 0.05215405
Iteration 30, loss = 0.05095524

### Let's test!

In [38]:
#
# how did it do? Try out the TEST data...
#

def ascii_table_for_regressor(Xsc,y,nn,scaler):
    """ a table including predictions using nn.predict """
    predictions = nn.predict(Xsc) # all predictions
    Xpr = scaler.inverse_transform(Xsc)  # Xpr is the "X to print": unscaled data!
    # measure error
    error = 0.0
    # printing
    print(f"{'input ':>35s} ->  {'pred':^6s}  {'des.':^6s}   {'absdiff':^10s}") 
    for i in range(len(y)):
        pred = predictions[i]
        desired = y[i]
        result = abs(desired - pred)
        error += result
        # Xpr = Xsc   # if you'd like to see the scaled values
        print(f"{Xpr[i,:]!s:>35s} ->  {pred:<+6.2f}  {desired:<+6.2f}   {result:^10.2f}") 

    print("\n" + "+++++   +++++   +++++           ")
    print(f"average abs diff error:   {error/len(y):<6.3f}")
    print("+++++   +++++   +++++           ")
    
#
# let's see how it did on the test data (also the training data!)
#
ascii_table_for_regressor(X_test_scaled,
                          y_test_scaled,
                          nn_regressor,
                          scaler)   # this is our own f'n, above


                             input  ->   pred    des.     absdiff  
          [5.4 1.3 0.4 1.  0.  0. ] ->  +3.79   +3.90       0.11   
          [4.7 1.6 0.2 1.  0.  0. ] ->  +3.10   +3.20       0.10   
          [4.3 1.1 0.1 1.  0.  0. ] ->  +2.89   +3.00       0.11   
          [6.3 4.4 1.3 0.  1.  0. ] ->  +2.84   +2.30       0.54   
          [4.6 1.5 0.2 1.  0.  0. ] ->  +3.04   +3.10       0.06   
          [5.5 4.  1.3 0.  1.  0. ] ->  +2.76   +2.50       0.26   
          [6.6 4.6 1.3 0.  1.  0. ] ->  +2.87   +2.90       0.03   
          [6.1 4.6 1.4 0.  1.  0. ] ->  +2.91   +3.00       0.09   
          [6.5 5.8 2.2 0.  0.  1. ] ->  +3.08   +3.00       0.08   
          [7.9 6.4 2.  0.  0.  1. ] ->  +3.07   +3.80       0.73   
          [6.8 5.5 2.1 0.  0.  1. ] ->  +3.03   +3.00       0.03   
          [5.8 3.9 1.2 0.  1.  0. ] ->  +2.71   +2.70       0.01   
          [7.1 5.9 2.1 0.  0.  1. ] ->  +3.06   +3.00       0.06   
          [5.7 3.5 1.  0.  1.  0. ] ->  +2.54   

## ***Petallen***

In [40]:
#
# Here we set up for a regression model that will predict 'sepallen'  (column index 0)
#
#   sepal length  'sepallen' (column index 0) 

# We will use
#
#   sepal width   'sepalwid' (column index 1) <-- our target for _regression_
#   petal length  'petallen' (column index 2)
#   petal width   'petalwid' (column index 3)
#   _and_
#   species       'irisnum'  (column index 4)    # No problem - we can use this as a feature!

print("+++ Start of data-assembly for feature-regression! +++\n")
# construct the correct X_all from the columns we want
# we use np.concatenate to combine parts of the dataset to get all-except-column 0:
#                     exclude 0  , include 1 to the end
X_all = np.concatenate( (A[:,0:2], A[:,3:]),axis=1)    # includes columns 1, 2, 3, and 4

# if we wanted all-except-column 1:   X_all = np.concatenate( (A[:,0:1], A[:,2:]),axis=1)  # columns 0, 2, 3, and 4
# if we wanted all-except-column 2:   X_all = np.concatenate( (A[:,0:2], A[:,3:]),axis=1)  # columns 0, 1, 3, and 4
# if we wanted all-except-column 3:   X_all = np.concatenate( (A[:,0:3], A[:,4:]),axis=1)  # columns 0, 1, 2, and 4
# if we wanted all-except-column 4:   X_all = np.concatenate( (A[:,0:4], A[:,5:]),axis=1)  # columns 0, 1, 2, and 3
# (slicing is forgiving)


y_all = A[:,2]                    # y (labels) ... is all of column 0, sepallen (sepal length) 
#                                 # change the line above to make other columns the target (y_all)
print(f"y_all is \n {y_all}")
print() 
print(f"X_all (the features, first few rows) is \n {X_all[:5,:]}")


+++ Start of data-assembly for feature-regression! +++

y_all is 
 [1.  1.1 1.2 1.2 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.4 1.4 1.4 1.4 1.4 1.4 1.4
 1.4 1.4 1.4 1.4 1.4 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.6
 1.6 1.6 1.6 1.6 1.6 1.7 1.7 1.7 1.7 1.9 1.9 4.7 3.3 3.3 3.5 3.5 3.6 3.7
 3.8 3.9 3.9 3.9 4.  4.  4.  4.  4.  4.1 4.1 4.2 4.2 4.2 4.3 4.4 4.4 4.4
 4.4 4.5 4.5 4.5 4.5 4.5 4.5 4.5 4.6 4.6 4.6 4.7 4.7 4.7 4.7 4.8 4.8 4.9
 4.9 5.  5.1 4.5 4.8 4.8 4.9 4.9 4.9 5.  5.  5.  5.1 5.1 5.1 5.1 5.1 5.1
 5.2 5.2 5.3 5.3 5.4 5.4 5.5 5.5 5.5 5.6 5.6 5.6 5.6 5.6 5.6 5.7 5.7 5.7
 5.8 5.8 5.8 5.9 5.9 6.  6.1 6.1 6.1 6.3 6.4 6.6 6.7 6.7 6.9]

X_all (the features, first few rows) is 
 [[4.6 3.6 0.2 1.  0.  0. ]
 [4.3 3.  0.1 1.  0.  0. ]
 [5.  3.2 0.2 1.  0.  0. ]
 [5.8 4.  0.2 1.  0.  0. ]
 [4.4 3.  0.2 1.  0.  0. ]]


### Permute!

In [41]:
#
# we scramble the data, to give a different TRAIN/TEST split each time...
# 
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_all = X_all[indices]              # we apply the _same_ permutation to each!
y_all = y_all[indices]              # again...
print("target values to predict: \n",y_all)
print("\nfeatures (a few)\n", X_all[0:5,:])

target values to predict: 
 [4.8 3.7 1.6 5.2 4.7 5.5 1.4 4.9 1.6 4.8 1.6 1.4 1.5 1.4 5.  4.5 6.7 5.6
 3.6 5.8 4.  6.4 5.7 4.5 4.7 5.6 1.7 1.4 3.5 1.7 4.5 1.6 5.1 5.2 4.  5.4
 3.5 4.8 1.3 1.5 1.5 1.3 5.3 5.  4.2 4.9 4.5 1.4 3.9 6.3 3.3 5.5 5.8 6.9
 5.4 4.2 5.9 1.5 4.5 1.3 1.3 5.1 4.3 6.1 4.  1.5 1.4 4.6 1.7 5.  1.4 4.7
 1.4 4.9 1.2 4.7 5.3 1.9 6.  4.6 1.5 1.6 6.6 4.  4.4 1.5 4.9 1.6 6.7 5.1
 1.4 3.9 5.5 4.9 4.1 5.6 1.5 5.7 5.6 4.5 1.2 4.5 5.9 4.7 4.5 3.8 1.9 4.8
 5.7 1.4 1.3 5.1 1.1 1.4 5.  5.1 1.5 5.1 5.6 1.7 4.4 5.1 6.1 3.3 1.  4.2
 4.4 4.6 6.1 5.6 1.4 1.3 3.9 1.5 1.5 5.8 1.3 4.  4.4 4.1 1.5]

features (a few)
 [[5.9 3.2 1.8 0.  1.  0. ]
 [5.5 2.4 1.  0.  1.  0. ]
 [4.7 3.2 0.2 1.  0.  0. ]
 [6.5 3.  2.  0.  0.  1. ]
 [6.1 2.8 1.2 0.  1.  0. ]]


### Split into training and testing sets

We'll stick with 80% training, 20% testing...

In [42]:
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2)

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

print(f"Held-out data... (testing data: {len(y_test)})")
print(f"y_test: {y_test}\n")
print(f"X_test (few rows): {X_test[0:5,:]}")  # 5 rows
print()
print(f"Data used for modeling... (training data: {len(y_train)})")
print(f"y_train: {y_train}\n")
print(f"X_train (few rows): {X_train[0:5,:]}")  # 5 rows

training with 112 rows;  testing with 29 rows

Held-out data... (testing data: 29)
y_test: [1.7 1.1 5.9 1.2 1.5 3.6 1.3 1.6 4.6 1.9 5.8 5.8 1.6 1.5 1.  4.5 1.7 5.1
 5.  1.5 1.5 4.4 5.6 6.9 1.4 5.6 1.6 5.7 1.4]

X_test (few rows): [[5.7 3.8 0.3 1.  0.  0. ]
 [4.3 3.  0.1 1.  0.  0. ]
 [6.8 3.2 2.3 0.  0.  1. ]
 [5.  3.2 0.2 1.  0.  0. ]
 [5.1 3.7 0.4 1.  0.  0. ]]

Data used for modeling... (training data: 112)
y_train: [1.4 4.  4.5 1.5 1.9 5.7 1.4 3.9 1.4 4.9 4.7 4.7 4.2 1.5 6.4 1.2 6.7 6.1
 4.5 5.1 4.5 3.5 5.  1.3 1.4 4.9 1.3 4.2 1.7 4.9 4.4 5.6 5.6 4.7 1.5 1.4
 1.5 1.3 1.4 4.5 1.4 5.9 1.6 4.8 5.4 1.3 4.  6.1 1.6 5.6 4.  5.3 5.1 4.4
 1.5 5.1 5.1 5.3 4.8 1.5 5.7 3.9 4.4 3.3 4.9 5.5 1.3 5.1 4.7 3.7 1.4 4.9
 5.8 4.  4.5 6.7 4.8 1.4 6.  5.5 5.  4.3 1.3 1.6 6.3 4.1 3.3 3.9 5.  1.5
 4.6 4.1 1.4 3.8 6.1 3.5 4.2 4.6 4.8 1.7 5.4 4.5 5.1 5.2 1.5 5.2 4.  4.7
 6.6 5.5 4.5 5.6]

X_train (few rows): [[4.6 3.4 0.3 1.  0.  0. ]
 [5.5 2.5 1.3 0.  1.  0. ]
 [6.  3.4 1.6 0.  1.  0. ]
 [4.9 3.1 0.1 1.  0

### It's a nnet - this time, we _should_ use a <u>Scaler</u>

In [46]:
#
# for NNets, it's important to keep the feature values near 0, say -1. to 1. or so
#    This is done through the "StandardScaler" in scikit-learn
#
from sklearn.preprocessing import StandardScaler

USE_SCALER = True   # this variable is important! It tracks if we need to use the scaler...

# we "train the scaler"  (computes the mean and standard deviation)
if USE_SCALER == True:
    scalerpl = StandardScaler()
    scalerpl.fit(X_train)  # Scale with the training data! ave becomes 0; stdev becomes 1
else:
    # this one does no scaling!  We still create it to be consistent:
    scalerpl = StandardScaler(copy=True, with_mean=False, with_std=False) # no scaling
    scalerpl.fit(X_train)  # still need to fit, though it does not change...

scalerpl   # is now defined and ready to use...

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++

# Here are our scaled training and testing sets:

X_train_scaled = scalerpl.transform(X_train) # scale!
X_test_scaled = scalerpl.transform(X_test) # scale!

y_train_scaled = y_train  # the predicted/desired labels are not scaled
y_test_scaled = y_test  # not using the scaler

# reused from above - seeing the scaled data 
ascii_table(X_train_scaled[0:5,:],y_train_scaled[0:5],None)

# reused from above - seeing the unscaled data (inverting the scaler)
ascii_table(X_train_scaled[0:5,:],y_train_scaled[0:5],scalerpl)

                                                    input  ->  pred   des  
                                 [-1.61  0.85 -1.34  1.62] ->    ?    1.4                  
                                 [-0.52 -1.19  0.05 -0.62] ->    ?    4.0                  
                                 [ 0.09  0.85  0.47 -0.62] ->    ?    4.5                  
                                 [-1.24  0.17 -1.62  1.62] ->    ?    1.5                  
                                 [-1.    1.75 -1.2   1.62] ->    ?    1.9                  

                                                    input  ->  pred   des  
                                         [4.6 3.4 0.3 1. ] ->    ?    1.4                  
                                         [5.5 2.5 1.3 0. ] ->    ?    4.0                  
                                         [6.  3.4 1.6 0. ] ->    ?    4.5                  
                                         [4.9 3.1 0.1 1. ] ->    ?    1.5                  
                   

### Create and train the model

In [47]:
#
# MLPRegressor predicts _floating-point_ outputs
#

from sklearn.neural_network import MLPRegressor

nn_regressor = MLPRegressor(hidden_layer_sizes=(6,7), 
                    max_iter=500,          # how many training epochs
                    verbose=True,          # do we want to watch as it trains?
                    shuffle=True,          # shuffle each epoch?
                    random_state=None,     # use for reproducibility
                    learning_rate_init=.1, # how much of each error to back-propagate
                    learning_rate = 'adaptive')  # how to handle the learning_rate

print("\n\n++++++++++  TRAINING:  begin  +++++++++++++++\n\n")
nn_regressor.fit(X_train_scaled, y_train_scaled)
print("++++++++++  TRAINING:   end  +++++++++++++++")
print(f"The (squared) prediction error (the loss) is {nn_regressor.loss_:<6.3f}")
print(f"And, its square root:         {nn_regressor.loss_ ** 0.5:<6.3f}")
print()



++++++++++  TRAINING:  begin  +++++++++++++++


Iteration 1, loss = 10.98463035
Iteration 2, loss = 8.48846880
Iteration 3, loss = 6.69305361
Iteration 4, loss = 5.01263736
Iteration 5, loss = 3.28755538
Iteration 6, loss = 1.98866681
Iteration 7, loss = 1.69293777
Iteration 8, loss = 1.48615209
Iteration 9, loss = 0.99024605
Iteration 10, loss = 0.33252026
Iteration 11, loss = 0.17423436
Iteration 12, loss = 0.38074885
Iteration 13, loss = 0.58127400
Iteration 14, loss = 0.61155183
Iteration 15, loss = 0.47359260
Iteration 16, loss = 0.28038311
Iteration 17, loss = 0.20447163
Iteration 18, loss = 0.26567075
Iteration 19, loss = 0.24991356
Iteration 20, loss = 0.15399816
Iteration 21, loss = 0.10458712
Iteration 22, loss = 0.13012175
Iteration 23, loss = 0.16984800
Iteration 24, loss = 0.17151999
Iteration 25, loss = 0.13541877
Iteration 26, loss = 0.08515558
Iteration 27, loss = 0.05401280
Iteration 28, loss = 0.05976825
Iteration 29, loss = 0.08776094
Iteration 30, loss = 0.1033794

### Let's test!

In [48]:
#
# how did it do? Try out the TEST data...
#

def ascii_table_for_regressor(Xsc,y,nn,scaler):
    """ a table including predictions using nn.predict """
    predictions = nn.predict(Xsc) # all predictions
    Xpr = scaler.inverse_transform(Xsc)  # Xpr is the "X to print": unscaled data!
    # measure error
    error = 0.0
    # printing
    print(f"{'input ':>35s} ->  {'pred':^6s}  {'des.':^6s}   {'absdiff':^10s}") 
    for i in range(len(y)):
        pred = predictions[i]
        desired = y[i]
        result = abs(desired - pred)
        error += result
        # Xpr = Xsc   # if you'd like to see the scaled values
        print(f"{Xpr[i,:]!s:>35s} ->  {pred:<+6.2f}  {desired:<+6.2f}   {result:^10.2f}") 

    print("\n" + "+++++   +++++   +++++           ")
    print(f"average abs diff error:   {error/len(y):<6.3f}")
    print("+++++   +++++   +++++           ")
    
#
# let's see how it did on the test data (also the training data!)
#
ascii_table_for_regressor(X_test_scaled,
                          y_test_scaled,
                          nn_regressor,
                          scaler)   # this is our own f'n, above


                             input  ->   pred    des.     absdiff  
          [5.7 3.8 0.3 1.  0.  0. ] ->  +1.55   +1.70       0.15   
          [4.3 3.  0.1 1.  0.  0. ] ->  +1.42   +1.10       0.32   
          [6.8 3.2 2.3 0.  0.  1. ] ->  +5.67   +5.90       0.23   
          [5.  3.2 0.2 1.  0.  0. ] ->  +1.39   +1.20       0.19   
          [5.1 3.7 0.4 1.  0.  0. ] ->  +1.53   +1.50       0.03   
          [5.6 2.9 1.3 0.  1.  0. ] ->  +3.95   +3.60       0.35   
          [4.7 3.2 0.2 1.  0.  0. ] ->  +1.41   +1.30       0.11   
          [4.7 3.2 0.2 1.  0.  0. ] ->  +1.41   +1.60       0.19   
          [6.5 2.8 1.5 0.  1.  0. ] ->  +4.79   +4.60       0.19   
          [4.8 3.4 0.2 1.  0.  0. ] ->  +1.42   +1.90       0.48   
          [6.5 3.  2.2 0.  0.  1. ] ->  +5.43   +5.80       0.37   
          [6.7 2.5 1.8 0.  0.  1. ] ->  +5.21   +5.80       0.59   
          [5.1 3.8 0.2 1.  0.  0. ] ->  +1.50   +1.60       0.10   
          [5.1 3.4 0.2 1.  0.  0. ] ->  +1.43   

## ***Petalwid***

In [49]:
#
# Here we set up for a regression model that will predict 'sepallen'  (column index 0)
#
#   sepal length  'sepallen' (column index 0) 

# We will use
#
#   sepal width   'sepalwid' (column index 1) <-- our target for _regression_
#   petal length  'petallen' (column index 2)
#   petal width   'petalwid' (column index 3)
#   _and_
#   species       'irisnum'  (column index 4)    # No problem - we can use this as a feature!

print("+++ Start of data-assembly for feature-regression! +++\n")
# construct the correct X_all from the columns we want
# we use np.concatenate to combine parts of the dataset to get all-except-column 0:
#                     exclude 0  , include 1 to the end
X_all = np.concatenate( (A[:,0:3], A[:,4:]),axis=1)    # includes columns 1, 2, 3, and 4

# if we wanted all-except-column 1:   X_all = np.concatenate( (A[:,0:1], A[:,2:]),axis=1)  # columns 0, 2, 3, and 4
# if we wanted all-except-column 2:   X_all = np.concatenate( (A[:,0:2], A[:,3:]),axis=1)  # columns 0, 1, 3, and 4
# if we wanted all-except-column 3:   X_all = np.concatenate( (A[:,0:3], A[:,4:]),axis=1)  # columns 0, 1, 2, and 4
# if we wanted all-except-column 4:   X_all = np.concatenate( (A[:,0:4], A[:,5:]),axis=1)  # columns 0, 1, 2, and 3
# (slicing is forgiving)


y_all = A[:,3]                    # y (labels) ... is all of column 0, sepallen (sepal length) 
#                                 # change the line above to make other columns the target (y_all)
print(f"y_all is \n {y_all}")
print() 
print(f"X_all (the features, first few rows) is \n {X_all[:5,:]}")


+++ Start of data-assembly for feature-regression! +++

y_all is 
 [0.2 0.1 0.2 0.2 0.2 0.2 0.3 0.2 0.3 0.4 0.2 0.2 0.3 0.2 0.1 0.3 0.2 0.2
 0.2 0.2 0.3 0.2 0.2 0.2 0.1 0.1 0.1 0.2 0.3 0.4 0.2 0.2 0.2 0.2 0.4 0.2
 0.2 0.2 0.4 0.6 0.2 0.5 0.4 0.2 0.3 0.2 0.4 1.4 1.  1.  1.  1.  1.3 1.
 1.1 1.4 1.1 1.2 1.3 1.3 1.2 1.  1.3 1.3 1.  1.3 1.2 1.5 1.3 1.2 1.3 1.4
 1.4 1.5 1.5 1.3 1.5 1.6 1.5 1.5 1.4 1.5 1.3 1.4 1.2 1.6 1.5 1.8 1.4 1.5
 1.5 1.7 1.6 1.7 1.8 1.8 2.  1.8 1.8 2.  1.5 1.9 2.4 1.9 1.8 1.5 2.  2.3
 2.  2.3 1.9 2.3 2.3 2.1 1.8 1.8 2.1 1.4 1.8 2.4 2.1 2.2 2.4 2.1 2.5 2.3
 2.2 1.8 1.6 2.3 2.1 1.8 2.5 1.9 2.3 1.8 2.  2.1 2.2 2.  2.3]

X_all (the features, first few rows) is 
 [[4.6 3.6 1.  1.  0.  0. ]
 [4.3 3.  1.1 1.  0.  0. ]
 [5.  3.2 1.2 1.  0.  0. ]
 [5.8 4.  1.2 1.  0.  0. ]
 [4.4 3.  1.3 1.  0.  0. ]]


### Permute!

In [50]:
#
# we scramble the data, to give a different TRAIN/TEST split each time...
# 
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_all = X_all[indices]              # we apply the _same_ permutation to each!
y_all = y_all[indices]              # again...
print("target values to predict: \n",y_all)
print("\nfeatures (a few)\n", X_all[0:5,:])

target values to predict: 
 [0.3 2.5 0.2 0.1 2.4 1.3 0.5 1.9 0.2 1.  2.  1.8 1.3 0.2 0.2 0.2 0.1 1.8
 2.3 0.4 0.2 0.4 1.3 1.8 0.2 0.3 1.3 2.3 1.5 2.1 1.3 1.6 1.  1.1 1.5 0.2
 1.5 1.8 2.  1.8 1.2 0.6 2.  1.5 0.4 1.4 1.  0.2 0.2 2.  2.1 1.5 0.2 1.9
 2.  0.1 2.  1.5 0.2 0.2 1.3 2.1 0.2 1.9 2.2 1.8 2.5 0.2 1.7 1.5 2.3 1.8
 0.4 0.2 1.4 1.9 1.2 1.6 1.1 2.2 1.5 1.6 1.8 0.2 1.8 1.8 2.3 0.3 0.4 2.3
 0.1 0.2 1.3 1.2 0.2 1.5 1.  1.8 2.4 0.2 1.5 1.4 1.5 1.8 2.1 0.2 1.  0.2
 0.1 2.4 2.2 0.2 0.2 1.2 1.4 2.3 0.3 1.4 1.3 1.4 0.3 2.1 0.2 1.3 0.3 1.2
 0.2 2.3 1.4 1.  1.5 1.4 1.7 1.6 2.1 1.3 2.3 0.4 0.2 1.  0.3]

features (a few)
 [[5.7 3.8 1.7 1.  0.  0. ]
 [7.2 3.6 6.1 0.  0.  1. ]
 [5.4 3.4 1.7 1.  0.  0. ]
 [4.8 3.  1.4 1.  0.  0. ]
 [5.8 2.8 5.1 0.  0.  1. ]]


### Split into training and testing sets

We'll stick with 80% training, 20% testing...

In [51]:
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2)

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

print(f"Held-out data... (testing data: {len(y_test)})")
print(f"y_test: {y_test}\n")
print(f"X_test (few rows): {X_test[0:5,:]}")  # 5 rows
print()
print(f"Data used for modeling... (training data: {len(y_train)})")
print(f"y_train: {y_train}\n")
print(f"X_train (few rows): {X_train[0:5,:]}")  # 5 rows

training with 112 rows;  testing with 29 rows

Held-out data... (testing data: 29)
y_test: [1.8 0.2 0.2 0.1 1.  1.2 1.5 2.1 1.3 0.2 2.  1.8 2.  1.9 0.3 0.1 1.5 1.9
 2.  1.4 0.2 1.3 1.7 1.5 1.  0.2 1.4 1.3 2.4]

X_test (few rows): [[5.9 3.  5.1 0.  0.  1. ]
 [4.6 3.6 1.  1.  0.  0. ]
 [4.4 3.2 1.3 1.  0.  0. ]
 [4.8 3.  1.4 1.  0.  0. ]
 [6.  2.2 4.  0.  1.  0. ]]

Data used for modeling... (training data: 112)
y_train: [0.4 0.2 0.2 2.5 2.2 1.9 1.4 0.6 1.  0.2 0.2 0.3 1.8 2.  0.2 2.3 0.2 0.3
 1.9 1.2 1.  1.  0.2 0.2 1.3 0.2 1.  1.6 0.2 1.6 2.3 1.5 0.2 2.2 1.4 2.3
 1.3 0.3 1.4 1.  1.5 0.5 0.4 0.2 1.5 0.2 2.1 2.1 0.2 0.2 0.2 1.8 2.  2.5
 1.8 1.3 1.4 0.2 2.1 1.8 1.6 2.3 1.2 0.2 1.3 1.2 0.2 1.5 1.4 0.4 2.3 0.2
 1.6 1.2 0.4 1.1 2.3 1.3 2.1 1.5 0.1 1.8 2.4 1.5 0.1 1.8 0.3 1.8 2.4 0.2
 1.8 1.5 1.8 2.1 1.4 1.5 0.4 0.2 1.7 0.3 1.5 1.1 1.8 1.3 2.3 2.2 0.1 2.
 1.3 0.3 0.4 2.3]

X_train (few rows): [[5.4 3.9 1.7 1.  0.  0. ]
 [5.8 4.  1.2 1.  0.  0. ]
 [5.  3.6 1.4 1.  0.  0. ]
 [7.2 3.6 6.1 0.  0.

### It's a nnet - this time, we _should_ use a <u>Scaler</u>

In [52]:
#
# for NNets, it's important to keep the feature values near 0, say -1. to 1. or so
#    This is done through the "StandardScaler" in scikit-learn
#
from sklearn.preprocessing import StandardScaler

USE_SCALER = True   # this variable is important! It tracks if we need to use the scaler...

# we "train the scaler"  (computes the mean and standard deviation)
if USE_SCALER == True:
    scalerpw = StandardScaler()
    scalerpw.fit(X_train)  # Scale with the training data! ave becomes 0; stdev becomes 1
else:
    # this one does no scaling!  We still create it to be consistent:
    scalerpw = StandardScaler(copy=True, with_mean=False, with_std=False) # no scaling
    scalerpw.fit(X_train)  # still need to fit, though it does not change...

scalerpw   # is now defined and ready to use...

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++

# Here are our scaled training and testing sets:

X_train_scaled = scalerpw.transform(X_train) # scale!
X_test_scaled = scalerpw.transform(X_test) # scale!

y_train_scaled = y_train  # the predicted/desired labels are not scaled
y_test_scaled = y_test  # not using the scaler

# reused from above - seeing the scaled data 
ascii_table(X_train_scaled[0:5,:],y_train_scaled[0:5],None)

# reused from above - seeing the unscaled data (inverting the scaler)
ascii_table(X_train_scaled[0:5,:],y_train_scaled[0:5],scalerpw)

                                                    input  ->  pred   des  
                                 [-0.55  1.87 -1.15  1.37] ->    ?    0.4                  
                                 [-0.06  2.1  -1.43  1.37] ->    ?    0.2                  
                                 [-1.04  1.2  -1.32  1.37] ->    ?    0.2                  
                                 [ 1.65  1.2   1.33 -0.73] ->    ?    2.5                  
                                 [ 2.26  1.65  1.67 -0.73] ->    ?    2.2                  

                                                    input  ->  pred   des  
                                         [5.4 3.9 1.7 1. ] ->    ?    0.4                  
                                         [5.8 4.  1.2 1. ] ->    ?    0.2                  
                                         [5.  3.6 1.4 1. ] ->    ?    0.2                  
                                         [7.2 3.6 6.1 0. ] ->    ?    2.5                  
                   

### Create and train the model

In [53]:
#
# MLPRegressor predicts _floating-point_ outputs
#

from sklearn.neural_network import MLPRegressor

nn_regressor = MLPRegressor(hidden_layer_sizes=(6,7), 
                    max_iter=500,          # how many training epochs
                    verbose=True,          # do we want to watch as it trains?
                    shuffle=True,          # shuffle each epoch?
                    random_state=None,     # use for reproducibility
                    learning_rate_init=.1, # how much of each error to back-propagate
                    learning_rate = 'adaptive')  # how to handle the learning_rate

print("\n\n++++++++++  TRAINING:  begin  +++++++++++++++\n\n")
nn_regressor.fit(X_train_scaled, y_train_scaled)
print("++++++++++  TRAINING:   end  +++++++++++++++")
print(f"The (squared) prediction error (the loss) is {nn_regressor.loss_:<6.3f}")
print(f"And, its square root:         {nn_regressor.loss_ ** 0.5:<6.3f}")
print()



++++++++++  TRAINING:  begin  +++++++++++++++


Iteration 1, loss = 1.45263480
Iteration 2, loss = 0.58969919
Iteration 3, loss = 0.40197305
Iteration 4, loss = 0.33882527
Iteration 5, loss = 0.16309599
Iteration 6, loss = 0.07753067
Iteration 7, loss = 0.08233695
Iteration 8, loss = 0.11515331
Iteration 9, loss = 0.12785931
Iteration 10, loss = 0.11889638
Iteration 11, loss = 0.09955503
Iteration 12, loss = 0.08097659
Iteration 13, loss = 0.06624929
Iteration 14, loss = 0.05790115
Iteration 15, loss = 0.05643443
Iteration 16, loss = 0.05607328
Iteration 17, loss = 0.04690065
Iteration 18, loss = 0.03958801
Iteration 19, loss = 0.03437104
Iteration 20, loss = 0.03944962
Iteration 21, loss = 0.04370269
Iteration 22, loss = 0.04665638
Iteration 23, loss = 0.04332589
Iteration 24, loss = 0.03581713
Iteration 25, loss = 0.02958332
Iteration 26, loss = 0.02557777
Iteration 27, loss = 0.02528472
Iteration 28, loss = 0.02658780
Iteration 29, loss = 0.02523818
Iteration 30, loss = 0.02228635

### Let's test!

In [54]:
#
# how did it do? Try out the TEST data...
#

def ascii_table_for_regressor(Xsc,y,nn,scaler):
    """ a table including predictions using nn.predict """
    predictions = nn.predict(Xsc) # all predictions
    Xpr = scaler.inverse_transform(Xsc)  # Xpr is the "X to print": unscaled data!
    # measure error
    error = 0.0
    # printing
    print(f"{'input ':>35s} ->  {'pred':^6s}  {'des.':^6s}   {'absdiff':^10s}") 
    for i in range(len(y)):
        pred = predictions[i]
        desired = y[i]
        result = abs(desired - pred)
        error += result
        # Xpr = Xsc   # if you'd like to see the scaled values
        print(f"{Xpr[i,:]!s:>35s} ->  {pred:<+6.2f}  {desired:<+6.2f}   {result:^10.2f}") 

    print("\n" + "+++++   +++++   +++++           ")
    print(f"average abs diff error:   {error/len(y):<6.3f}")
    print("+++++   +++++   +++++           ")
    
#
# let's see how it did on the test data (also the training data!)
#
ascii_table_for_regressor(X_test_scaled,
                          y_test_scaled,
                          nn_regressor,
                          scalerpw)   # this is our own f'n, above


                             input  ->   pred    des.     absdiff  
          [5.9 3.  5.1 0.  0.  1. ] ->  +2.15   +1.80       0.35   
          [4.6 3.6 1.  1.  0.  0. ] ->  +0.30   +0.20       0.10   
          [4.4 3.2 1.3 1.  0.  0. ] ->  +0.26   +0.20       0.06   
          [4.8 3.  1.4 1.  0.  0. ] ->  +0.21   +0.10       0.11   
          [6.  2.2 4.  0.  1.  0. ] ->  +1.32   +1.00       0.32   
          [5.8 2.6 4.  0.  1.  0. ] ->  +1.32   +1.20       0.12   
          [5.9 3.  4.2 0.  1.  0. ] ->  +1.32   +1.50       0.18   
          [7.6 3.  6.6 0.  0.  1. ] ->  +2.09   +2.10       0.01   
          [5.5 2.5 4.  0.  1.  0. ] ->  +1.32   +1.30       0.02   
          [4.8 3.4 1.6 1.  0.  0. ] ->  +0.26   +0.20       0.06   
          [6.5 3.  5.2 0.  0.  1. ] ->  +2.07   +2.00       0.07   
          [6.7 2.5 5.8 0.  0.  1. ] ->  +1.79   +1.80       0.01   
          [5.7 2.5 5.  0.  0.  1. ] ->  +1.80   +2.00       0.20   
          [7.4 2.8 6.1 0.  0.  1. ] ->  +1.99   

The abs.error are: 
+ sepallen: 0.214
+ sepalwid: 0.221
+ petallen: 0.254
+ petalwid: 0.133
Therefore, petalwid is more predictable compare to other features because it has the smallest abs.error

In [ ]:
"""
Your task!

Just as above, find the average abs. error in the other three botanical features:
  + sepalwid
  + petallen
  + petalwid
  + above is sepallen!
  
Looping option:
  + run a loop over the columns... (see the concatenate call, above)
  + When you use concatenate, be sure all are _slices_, not single columns
  
Copy-and-paste option:
  + copy-and-paste a lot!
  + You will feel like regressing is second nature (this isn't a bad thing!)
  + You might also feel you've regressed... if so, try the looping approach, instead!
  
Then, include a text (or markdown) cell with your _four_ av. abs. errors on the TEST set
  + this is one average absolute error for each of the four botanical features

Just as some features are more important than others,
  + so, too are some features more _predictable_ than others...
  
Not required...  but interesting:
  How much do different network-shapes matter here?

Not required... but also interesting:  
  You could _then_ drop the species column: species is now a feature! 

  ... in order to estimate how much "value" is added, by knowing the flower-species, 
  when predicting each botanical measurement...

"""
42